In [1]:
event={'Records': [{'eventVersion': '2.1', 'eventSource': 'aws:s3', 'awsRegion': 'us-east-2', 
'eventTime': '2023-03-05T10:30:12.206Z', 'eventName': 'ObjectCreated:Put', 'userIdentity': {'principalId': 'A2A12KSW4I0LET'}, 
'requestParameters': {'sourceIPAddress': '49.37.219.143'}, 'responseElements': 
{'x-amz-request-id': 'TDZ3MYW5BYTX7V4W', 'x-amz-id-2': 'jvHk3bP57wQagxMm2nSAFUuSw3FYxTBa9YFKUzOs0mgFe1rU/Uu50V9RJ7gxz45CMNVhI+sYwPDcO0WiE6meqNlnoWe/YsFJloTznfuTurk='}, 's3': {'s3SchemaVersion': '1.0', 'configurationId': '91f4695e-452f-45d4-a736-e4af29f0c9a2', 'bucket': {'name': 'data-analytics-ck', 'ownerIdentity': {'principalId': 'A2A12KSW4I0LET'}, 'arn': 'arn:aws:s3:::data-analytics-ck'}, 'object': {'key': 'csvstorage/CROPSINDIA2002-2022.csv', 'size': 35898, 'eTag': 'e81a295b27dcbe0b5ee56a89422a4cd1', 'sequencer': '0064046F34237DC21B'}}}]}

In [2]:
event

{'Records': [{'eventVersion': '2.1',
   'eventSource': 'aws:s3',
   'awsRegion': 'us-east-2',
   'eventTime': '2023-03-05T10:30:12.206Z',
   'eventName': 'ObjectCreated:Put',
   'userIdentity': {'principalId': 'A2A12KSW4I0LET'},
   'requestParameters': {'sourceIPAddress': '49.37.219.143'},
   'responseElements': {'x-amz-request-id': 'TDZ3MYW5BYTX7V4W',
    'x-amz-id-2': 'jvHk3bP57wQagxMm2nSAFUuSw3FYxTBa9YFKUzOs0mgFe1rU/Uu50V9RJ7gxz45CMNVhI+sYwPDcO0WiE6meqNlnoWe/YsFJloTznfuTurk='},
   's3': {'s3SchemaVersion': '1.0',
    'configurationId': '91f4695e-452f-45d4-a736-e4af29f0c9a2',
    'bucket': {'name': 'data-analytics-ck',
     'ownerIdentity': {'principalId': 'A2A12KSW4I0LET'},
     'arn': 'arn:aws:s3:::data-analytics-ck'},
    'object': {'key': 'csvstorage/CROPSINDIA2002-2022.csv',
     'size': 35898,
     'eTag': 'e81a295b27dcbe0b5ee56a89422a4cd1',
     'sequencer': '0064046F34237DC21B'}}}]}

In [3]:
bucketName=event["Records"][0]["s3"]["bucket"]["name"]
key=event["Records"][0]["s3"]["object"]["key"]

In [4]:
bucketName,key

('data-analytics-ck', 'csvstorage/CROPSINDIA2002-2022.csv')

In [1]:
import boto3
boto3.session.Session().available_profiles

#boto3.setup_default_session(profile_name='ck-pythonist')

['default',
 'chandrakanthOI',
 'chandrakanthPythonist',
 'pythonist',
 'ck-pythonist']

In [2]:
boto3.setup_default_session(profile_name='ck-pythonist')

In [3]:
from lambda_function import *

NoRegionError: You must specify a region.

In [2]:
transactions=[{"transactionid":"trch20230319223","customerid":"chandrakanth",
               "item":"Bread","qty":25,"rate":50,"amount":1250},
               {"transactionid":"trch20230319224","customerid":"chandrakanth",
               "item":"Jam","qty":5,"rate":50,"amount":250},
               {"transactionid":"trch20230319225","customerid":"kp",
               "item":"Bread","qty":250,"rate":50,"amount":12500},
               {"transactionid":"trch20230319226","customerid":"kp",
               "item":"Jam","qty":5,"rate":50,"amount":250},
               ]

In [3]:
transactionData=pd.DataFrame.from_dict(transactions,orient="columns")

In [4]:
transactionData

,transactionid,customerid,item,qty,rate,amount
0,trch20230319223,chandrakanth,Bread,25,50,1250
1,trch20230319224,chandrakanth,Jam,5,50,250
2,trch20230319225,kp,Bread,250,50,12500
3,trch20230319226,kp,Jam,5,50,250


In [5]:
dynamoDBJson=CSVtoDynamoDBJson(transactionData)

In [6]:
dynamoDBJson

[[{'transactionid': {'S': 'trch20230319223'},
   'customerid': {'S': 'chandrakanth'},
   'item': {'S': 'Bread'},
   'qty': {'N': '25'},
   'rate': {'N': '50'},
   'amount': {'N': '1250'}},
  {'transactionid': {'S': 'trch20230319224'},
   'customerid': {'S': 'chandrakanth'},
   'item': {'S': 'Jam'},
   'qty': {'N': '5'},
   'rate': {'N': '50'},
   'amount': {'N': '250'}}],
 [{'transactionid': {'S': 'trch20230319225'},
   'customerid': {'S': 'kp'},
   'item': {'S': 'Bread'},
   'qty': {'N': '250'},
   'rate': {'N': '50'},
   'amount': {'N': '12500'}},
  {'transactionid': {'S': 'trch20230319226'},
   'customerid': {'S': 'kp'},
   'item': {'S': 'Jam'},
   'qty': {'N': '5'},
   'rate': {'N': '50'},
   'amount': {'N': '250'}}]]

In [13]:
key="TransactionFiles/Transactions20230319.csv"
bucketName="data-analytics-ck"
queueURL="https://sqs.us-east-2.amazonaws.com/073914561898/DataAnalytics"

In [14]:
sqs=boto3.client("sqs",region_name="us-east-2")

In [6]:
sqs.list_queues()

{'QueueUrls': ['https://sqs.us-east-2.amazonaws.com/073914561898/DataAnalytics',
  'https://sqs.us-east-2.amazonaws.com/073914561898/DataAnalytics.fifo'],
 'ResponseMetadata': {'RequestId': '955cbf6d-7a6a-5011-931d-e3f503ce24fe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '955cbf6d-7a6a-5011-931d-e3f503ce24fe',
   'date': 'Sat, 25 Mar 2023 04:24:04 GMT',
   'content-type': 'text/xml',
   'content-length': '417'},
  'RetryAttempts': 0}}

In [17]:
sqs.send_message(QueueUrl=queueURL,
    MessageBody='abcdefg',
    MessageAttributes={
        'ABCD': {
            'StringValue': 'ABCDEFG',
            'DataType': 'String'
        }
    })

{'MD5OfMessageBody': '7ac66c0f148de9519b8bd264312c4d64',
 'MD5OfMessageAttributes': '5fae4b32871c46be54fc216d96a671a5',
 'MessageId': '9bbd2b0c-c611-42ed-bd51-ecadd095be1e',
 'ResponseMetadata': {'RequestId': 'ec848ae3-ed22-5052-a10c-d67b92bb70bd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ec848ae3-ed22-5052-a10c-d67b92bb70bd',
   'date': 'Sat, 25 Mar 2023 04:49:29 GMT',
   'content-type': 'text/xml',
   'content-length': '459'},
  'RetryAttempts': 0}}